In [1]:
import sys,json,gzip,random,re,math
from collections import Counter
import numpy as np
from sklearn.cross_validation import KFold
from sklearn import linear_model
from scipy import sparse
from sklearn.preprocessing import normalize
from sklearn import cross_validation
from sklearn.grid_search import GridSearchCV

vocab={}
liwc={}
liwc_vocab={}
regex_liwc={}

/home/jrgillick/.local/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/jrgillick/.local/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [10]:
# get all flat LIWC categories for a word
def getLIWC(word):
    vals=[]
    if word in liwc:
        vals.extend(liwc[word])
    if len(word) > 1:
        pref=word[0:2]
        if pref in regex_liwc:
            cands=regex_liwc[pref]
            for cand in cands:
                if re.match(cand, word) != None:
                    vals.extend(regex_liwc[pref][cand])
    return vals

def readLIWC(filename):
    # Liu excludes "affective or emotional processes"
    invalid={}
    invalid["31"]=1
    invalid["32"]=1
    invalid["33"]=1
    invalid["34"]=1
    invalid["35"]=1

    file=open(filename)
    for i in range(75):
        line=file.readline()
        cols=re.split("\s+", line.rstrip())
        if line.rstrip() != "%":
            idd="%s" % cols[0]
            label=cols[1]
            liwc_vocab[idd]=label
    for line in file:
        cols=line.rstrip().split("\t")
        term=cols[0]
        valid=[]
        for x in cols[1:]:
            if x not in invalid:
                valid.append(x)
        cats=["LIWC_%s" % liwc_vocab[x] for x in valid]

        if term.endswith("*"):
            pref=term[0:2]
            if pref not in regex_liwc:
                regex_liwc[pref]={}
            regex_liwc[pref][term]=cats
        else:
            liwc[term]=cats

In [11]:
N=0
fid=1

In [15]:
filename = transcripts[0]
filename

'/data/corpora/ted/transcripts_clean/a_robot_that_flies_like_a_bird.html'

In [12]:
def read(filename):

    N=0
    fid=1

    for key in liwc_vocab:
        feat="LIWC_%s" % (liwc_vocab[key])
        vocab[feat]=fid
        fid+=1

    counts=Counter()
    f=open(filename)
    for line in f:
        cols=line.rstrip().split("\t")
        label=cols[0]
        text=cols[1].lower().split(" ")
        N+=1
        for word in text:
            counts[word.lower()]+=1
    f.close()
    for word in counts:
        count=counts[word]
        if count >= 500000:
            vocab[word.lower()]=fid
            fid+=1


    X = sparse.dok_matrix((N+1, fid+1))	
    Y = np.zeros(N+1)

    f=open(filename)
    docid=1
    for line in f:
        cols=line.rstrip().split("\t")
        label=int(cols[0])
        text=cols[1].lower().split(" ")

        for word in text:
            cats=getLIWC(word)
            for cat in cats:
                X[docid,vocab[cat]]=1
        for word in text:
            if word in vocab:
                X[docid,vocab[word]]=1
        Y[docid]=label
        docid+=1
    f.close()

    return X, Y

In [7]:
readLIWC('/data/corpora/LIWC/LIWC.txt')

In [8]:
import os

pathname="/data/corpora/ted/transcripts_clean/"
files = [pathname + f for f in os.listdir(pathname)]
talk_names = open('/home/jrgillick/Applause/talk_names.txt').read().replace('.txt','.html').split('\n')
talk_names = ['isabel_allende_how_to_live_passionately_no_matter_your_age.html' if t == 'isabelle_allende_how_to_live_passionately_no_matter_your_age.html' else t for t in talk_names
]
talk_names = [t for t in talk_names if t not in ['', 'test.html', '\r']]
transcripts = [pathname + t for t in talk_names]
len(transcripts)

904

In [13]:
read(transcripts[0])

ValueError: invalid literal for int() with base 10: '0:11'